### Live video Test for CNN_Gesture Recognition 

In [1]:
from tensorflow.keras.models import load_model
import cv2
import numpy as np
import tensorflow as tf

In [2]:
def get_class_label(pred_idx):
    labels = ['LeftSwipe', 'RightSwipe', 'Stop', 'ThumbsDown', 'ThumbsUp'] # Customize based on labels
    return labels[pred_idx]

def loadModel():# load the existing model
    model = load_model(modelToLoad)
    model.summary()

    return model

def cropResize(image, y, z):
    h, w = image.shape

    # if smaller image crop at center for 120x120
    if w == 160:
        image = image[:120, 20:140]

    # resize every image
    return resize(image, (y,z))

def normalizeImage(image):
    # applying normalization
    return image/255.0  

def preprocessImage(image, y, z):
    return normalizeImage(cropResize(image, y, z))


def main():
    #print("Start*******")
    # load the model
    model = loadModel()

    #print("Model Load Done*******")
    #model.summary()

    #infer = model.signatures["serving_default"]

    # Initialize webcam
    cap = cv2.VideoCapture(0)  # '0' is typically the default value for the webcam
 
    if not cap.isOpened():
        print("Error: Could not open webcam.")
        exit()
    
    # Buffer to hold frames
    frame_buffer = []
    
    window_width = 1280
    window_height = 600
    # Initialize video writer
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter('output.avi', fourcc, 20.0, (window_width, window_height))


    #c = 0
    # Read frames from the webcam
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            print("Failed to grab frame")
            break

        # Preprocess the frame
        frame_resized = cv2.resize(frame, input_size)
        frame_rgb = cv2.cvtColor(frame_resized, cv2.COLOR_BGR2RGB)
        frame_processed = frame_rgb.astype(np.float32) / 255.0

        #print("Shape of Processed Frame:::", frame_processed.shape)
        # Convert frame to float32
        #frame_processed = frame_processed.astype(np.float32) / 255

        # Add the frame to the buffer
        frame_buffer.append(frame_processed)

        # Maintain only the last 20 frames
        if len(frame_buffer) > sequence_length:
            frame_buffer.pop(0)

        #rgb_frame = cv2.cvtColor(frame_processed, cv2.COLOR_BGR2RGB)
        
        if len(frame_buffer) == sequence_length:
            input_sequence = np.array(frame_buffer)
            input_tensor = tf.convert_to_tensor(input_sequence, dtype=tf.float32)
            #input_tensor = tf.image.per_image_standardization(input_tensor)
            input_tensor = tf.expand_dims(input_tensor, axis=0)

            #print("*************************",input_tensor.shape)

            # Perform inference
            #predictions = infer(input_tensor)
            predictions = model.predict(input_tensor)
            #print(predictions)
            #output = predictions["dense"].numpy()
            output = predictions

            
            #print("******** Predicitons and prediction label***********")
            #print(output)
            label = np.argmax(output, axis=-1)
            #print("Label Data::", label)
            labelText = get_class_label(label[0])
            #print("Prediction Label:::",labelText)

            label_text = f"Prediction: {labelText}"
            cv2.putText(frame, label_text, (20, 60), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)


 
        # Resize frame for display window
        display_frame = cv2.resize(frame, (window_width, window_height))
        
        # Write the frame to the video file
        #out.write(display_frame)
        
        # Display the resulting frame
        #cv2.imshow('Video', cv2.cvtColor(frame, cv2.COLOR_GRAY2BGR))
        cv2.imshow('Video', display_frame)
        
 
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

if __name__ == '__main__':

    input_size = (128, 128)
    sequence_length = 30
    #modelToLoad = '/Users/jiten/Masters/Compute vision - CSC 528/CNN_Gesture_recognition/mobilenet_lstm_2024-06-0100_04_37.226216/model-keras.keras'
    modelToLoad = 'mobilenet_lstm_2024-06-0116_10_00.592795/model-keras.keras'
    #modelToLoad = 'mobilenet_lstm_2024-06-0100_04_37.226216/model-keras.keras'
    main()

Model: "mobilenet_lstm"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_44 (TimeDi  (None, 30, 4, 4, 1024)   3228864   
 stributed)                                                      
                                                                 
 time_distributed_45 (TimeDi  (None, 30, 4, 4, 1024)   4096      
 stributed)                                                      
                                                                 
 time_distributed_46 (TimeDi  (None, 30, 2, 2, 1024)   0         
 stributed)                                                      
                                                                 
 time_distributed_47 (TimeDi  (None, 30, 4096)         0         
 stributed)                                                      


2024-06-03 16:27:56.547895: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2024-06-03 16:27:56.549375: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2024-06-03 16:27:56.549650: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

                                                                 
 lstm_4 (LSTM)               (None, 256)               4457472   
                                                                 
 dropout_17 (Dropout)        (None, 256)               0         
                                                                 
 dense_12 (Dense)            (None, 5)                 1285      
                                                                 
Total params: 7,691,717
Trainable params: 7,667,781
Non-trainable params: 23,936
_________________________________________________________________


[ WARN:0@29.301] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (862) isPipelinePlaying OpenCV | GStreamer warning: GStreamer: pipeline have not been created
2024-06-03 16:27:56.948 python[13423:3289186] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
[ WARN:0@30.785] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap_gstreamer.cpp (2180) open OpenCV | GStreamer warning: cannot link elements
[ERROR:0@30.792] global /private/var/folders/k1/30mswbxs7r1g6zwn8y4fyt500000gp/T/abs_11nitadzeg/croot/opencv-suite_1691620374638/work/modules/videoio/src/cap.cpp (597) open VIDEOIO(CV_IMAGES): raised OpenCV exception:

OpenCV(4.6.0) /private/var/folders/k1/30mswbxs7r

1/1 [==============================] - 0s 59ms/step


...

1/1 [==============================] - 0s 74ms/step


1/1 [==============================] - 0s 74ms/step


1/1 [==============================] - 0s 63ms/step


1/1 [==============================] - 0s 64ms/step


1/1 [==============================] - 0s 71ms/step
